# CPB - Cryptocurrency Price Prediction with LSTM (Google Colab)

此筆記本用於在Google Colab上訓練加密貨幣價格預測模型。

## 功能
- 自動掛載Google Drive
- Binance API 資料採集 (20+ 幣種)
- 35+ 技術指標特徵工程
- PyTorch LSTM 模型訓練 (使用免費GPU)
- 模型評估與性能報告

## 前置要求
1. Google Colab 帳號 (免費)
2. Google Drive (用於儲存資料與模型)
3. Binance API 密鑰 (可選，無密鑰只能使用公開資料)


## Step 1: 掛載Google Drive

In [ ]:
from google.colab import drive
import os

# 掛載Google Drive
drive.mount('/content/drive')

# 建立工作目錄
work_dir = '/content/drive/MyDrive/cpb_training'
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)

print(f'工作目錄: {os.getcwd()}')
print(f'可用GPU: {os.system("nvidia-smi")}')  # 檢查GPU

## Step 2: Clone GitHub Repo

In [ ]:
import subprocess

# Clone repo
if not os.path.exists('cpb'):
    subprocess.run(['git', 'clone', 'https://github.com/caizongxun/cpb.git'], cwd=work_dir)
else:
    # 更新已存在的repo
    subprocess.run(['git', 'pull'], cwd=os.path.join(work_dir, 'cpb'))

os.chdir(os.path.join(work_dir, 'cpb'))
print(f'當前目錄: {os.getcwd()}')
print(f'檔案列表: {os.listdir()}')

## Step 3: 安裝依賴

In [ ]:
# 安裝requirements
!pip install -q -r requirements.txt

print('依賴安裝完成！')

# 驗證PyTorch安裝
import torch
print(f'PyTorch版本: {torch.__version__}')
print(f'CUDA可用: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU型號: {torch.cuda.get_device_name(0)}')

## Step 4: 設定 .env 檔案

In [ ]:
# 建立 .env 檔案（如果不存在）
import os

env_content = """# Binance API Configuration
BINANCE_API_KEY=your_api_key_here
BINANCE_API_SECRET=your_api_secret_here

# Model Configuration
MODEL_NAME=lstm_crypto_predictor
LOOKBACK_PERIOD=60
NUM_FEATURES=30
BATCH_SIZE=32
EPOCHS=150
LEARNING_RATE=0.001

# Data Configuration
DATA_DIR=./data
MODEL_DIR=./models
TIMEFRAMES=15m,1h
NUM_COINS=20+

# CUDA Configuration
CUDA_VISIBLE_DEVICES=0
USE_CUDA=true
"""

with open('.env', 'w') as f:
    f.write(env_content)

print('.env 檔案已建立！')
print('
提示: 如果有Binance API密鑰，請修改.env檔案中的BINANCE_API_KEY與BINANCE_API_SECRET')

## Step 5: 運行完整訓練Pipeline

此步驟會自動執行：
1. **Phase 1**: Binance API 資料採集 (20+ 幣種，1h & 15m)
2. **Phase 2**: 特徵工程 (35+ 技術指標)
3. **Phase 3**: LSTM 模型訓練 (使用GPU加速)
4. **Phase 4**: 模型評估與報告

**預計耗時**: 2-4 小時（取決於GPU與幣種數量）

In [ ]:
import sys
sys.path.insert(0, './src')

from train import CryptoMLPipeline

# 建立Pipeline實例
pipeline = CryptoMLPipeline(is_colab=True)

# 執行完整Pipeline
results_df = pipeline.run_full_pipeline(
    days_back=30,              # 採集30天資料
    epochs=150,                # 訓練150個epoch
    skip_collection=False      # 改為True可跳過採集，使用本地資料
)

print('\n訓練完成！')

## Step 6: 檢視訓練結果

In [ ]:
import pandas as pd
import json

# 顯示結果DataFrame
print('=== 訓練結果摘要 ===')
print(results_df)

# 讀取詳細報告
with open('./models/training_report.json', 'r') as f:
    report = json.load(f)

print('\n=== 性能統計 ===')
summary = report['summary']
print(f"訓練幣種數: {summary['total_coins']}")
print(f"平均 RMSE: {summary['avg_rmse']:.6f}")
print(f"平均 MAPE: {summary['avg_mape']:.2f}%")
print(f"平均方向準確率: {summary['avg_direction_accuracy']:.2f}%")

## Step 7: 下載模型與報告

In [ ]:
import os
from pathlib import Path

# 檢查輸出檔案
model_dir = Path('./models')
data_dir = Path('./data')

print('=== 已生成的模型 ===')
for model_file in sorted(model_dir.glob('*.pt')):
    size_mb = model_file.stat().st_size / (1024*1024)
    print(f'{model_file.name}: {size_mb:.2f} MB')

print('\n=== 報告檔案 ===')
for report_file in model_dir.glob('*.json'):
    print(f'{report_file.name}')

print('\n=== 原始資料 ===')
raw_data_dir = data_dir / 'raw_data'
if raw_data_dir.exists():
    csv_files = list(raw_data_dir.glob('*.csv'))
    print(f'CSV檔案數: {len(csv_files)}')

## Step 8 (可選): 繪製訓練曲線

In [ ]:
import matplotlib.pyplot as plt
import json

# 如果有保存訓練日誌，可以繪製損失曲線
# 這需要在model_trainer.py中額外保存training logs

# 暫時顯示結果統計
results_sorted = results_df.sort_values('direction_accuracy', ascending=False)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# RMSE
axes[0, 0].hist(results_df['rmse'], bins=10, color='skyblue', edgecolor='black')
axes[0, 0].set_title('RMSE Distribution')
axes[0, 0].set_xlabel('RMSE')
axes[0, 0].set_ylabel('Frequency')

# MAPE
axes[0, 1].hist(results_df['mape'], bins=10, color='lightcoral', edgecolor='black')
axes[0, 1].set_title('MAPE Distribution')
axes[0, 1].set_xlabel('MAPE (%)')
axes[0, 1].set_ylabel('Frequency')

# Direction Accuracy
axes[1, 0].hist(results_df['direction_accuracy'], bins=10, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Direction Accuracy Distribution')
axes[1, 0].set_xlabel('Accuracy (%)')
axes[1, 0].set_ylabel('Frequency')

# Top 10 Coins
top_coins = results_sorted.head(10)
axes[1, 1].barh(range(len(top_coins)), top_coins['direction_accuracy'])
axes[1, 1].set_yticks(range(len(top_coins)))
axes[1, 1].set_yticklabels(top_coins.index)
axes[1, 1].set_title('Top 10 Coins by Direction Accuracy')
axes[1, 1].set_xlabel('Direction Accuracy (%)')

plt.tight_layout()
plt.savefig('./models/training_results.png', dpi=100, bbox_inches='tight')
plt.show()

print('圖表已儲存: ./models/training_results.png')

## 常見問題

### Q1: Colab 中斷了怎麼辦？
- 模型已自動保存在Google Drive中
- 下次執行時，改`skip_collection=True`跳過資料採集

### Q2: 如何使用自己的API密鑰？
- 在Step 4修改.env檔案中的`BINANCE_API_KEY`和`BINANCE_API_SECRET`

### Q3: 訓練太慢怎麼辦？
- 減少幣種數量 (修改`data_collector.py`中的`self.coins`)
- 減少`epochs`參數
- 增加`batch_size`

### Q4: 如何只訓練某個幣種？
- 修改`data_collector.py`中的幣種清單
- 只保留想要的幣種

### Q5: GPU不被使用怎麼辦？
- 在Colab中進入`設定 > 筆記本設定`
- 改GPU為'T4'或'L4'
- 重新執行Step 3驗證GPU可用性